# Data Cleaning Notebook

## Objectives

*   Evaluate missing data
*   Clean data

## Inputs

* outputs/datasets/collection/SalePrice.csv

## Outputs

* Generate cleaned Train and Test sets, both saved under outputs/datasets/cleaned

## Conclusions

 
  * Data Cleaning Pipeline
 



---


# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/HeritageHousing/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/HeritageHousing'

---

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps
import warnings
%matplotlib inline

warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from ydata_profiling import ProfileReport
from feature_engine.imputation import CategoricalImputer
from feature_engine.imputation import MeanMedianImputer


# Load Collected data

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
import pandas as pd
df = pd.read_csv("outputs/datasets/HousePricing.csv")
df.head(20)

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,0.0,548,RFn,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,RFn,...,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,RFn,...,68.0,162.0,42,5,7,920,NaN,2001,2002,223500
3,961,NaN,NaN,No,216,ALQ,540,NaN,642,Unf,...,60.0,0.0,35,5,7,756,NaN,1915,1970,140000
4,1145,NaN,4.0,Av,655,GLQ,490,0.0,836,RFn,...,84.0,350.0,84,5,8,1145,NaN,2000,2000,250000
5,796,566.0,1.0,No,732,GLQ,64,NaN,480,Unf,...,85.0,0.0,30,5,5,796,NaN,1993,1995,143000
6,1694,0.0,3.0,Av,1369,GLQ,317,NaN,636,RFn,...,75.0,186.0,57,5,8,1686,NaN,2004,2005,307000
7,1107,983.0,3.0,Mn,859,ALQ,216,NaN,484,NaN,...,NaN,240.0,204,6,7,1107,NaN,1973,1973,200000
8,1022,752.0,2.0,No,0,Unf,952,NaN,468,Unf,...,51.0,0.0,0,5,7,952,NaN,1931,1950,129900
9,1077,0.0,2.0,No,851,GLQ,140,NaN,205,RFn,...,50.0,0.0,4,6,5,991,NaN,1939,1950,118000


# Data Exploration

In Data Cleaning you are interested to check the distribution and shape of a variable with missing data.

In [ ]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

In [ ]:
from ydata_profiling import ProfileReport
if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")


# Data Cleaning

## Assessing Missing Data Levels

* Custom function to display missing data levels in a DataFrame, it shows the absolute levels, relative levels and data type.

In [ ]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (pd.DataFrame(
                            data={"RowsWithMissingData": missing_data_absolute,
                                   "PercentageOfDataset": missing_data_percentage,
                                   "DataType": df.dtypes}
                                    )
                          .sort_values(by=['PercentageOfDataset'], ascending=False)
                          .query("PercentageOfDataset > 0")
                          )

    return df_missing_data


Check missing data levels for the collected dataset.

In [ ]:
EvaluateMissingData(df)

## Dealing with Missing Data

* This is a custom function from the 'Feature Engineering Lesson Notebook unit 9'

In [ ]:
def DataCleaningEffect(df_original,df_cleaned,variables_applied_with_method):

  flag_count=1 # Indicate plot number
  
  # distinguish between numerical and categorical variables
  categorical_variables = df_original.select_dtypes(exclude=['number']).columns 

  # scan over variables, 
    # first on variables that you applied the method
    # if the variable is a numerical plot, a histogram if categorical plot a barplot
  for set_of_variables in [variables_applied_with_method]:
    print("\n=====================================================================================")
    print(f"* Distribution Effect Analysis After Data Cleaning Method in the following variables:")
    print(f"{set_of_variables} \n\n")
  

    for var in set_of_variables:
      if var in categorical_variables:  # it is categorical variable: barplot
        
        df1 = pd.DataFrame({"Type":"Original","Value":df_original[var]})
        df2 = pd.DataFrame({"Type":"Cleaned","Value":df_cleaned[var]})
        dfAux = pd.concat([df1, df2], axis=0)
        fig , axes = plt.subplots(figsize=(15, 5))
        sns.countplot(hue='Type', data=dfAux, x="Value",palette=['#432371',"#FAAE7B"])
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.xticks(rotation=90)
        plt.legend() 

      else: # it is numerical variable: histogram

        fig , axes = plt.subplots(figsize=(10, 5))
        sns.histplot(data=df_original, x=var, color="#432371", label='Original', kde=True,element="step", ax=axes)
        sns.histplot(data=df_cleaned, x=var, color="#FAAE7B", label='Cleaned', kde=True,element="step", ax=axes)
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.legend() 

      plt.show()
      flag_count+= 1

### Data Cleaning Summary

Potential Cleaning approaches 

* Median Imputer - `['2ndFlrSF', 'BedroomAbvGr', 'GarageYrBlt', 'LotFrontage', 'MasVnrArea']` since the distribution isn't bell shape curved

* Categorical Imputer - `['BsmtFinType1','GarageFinish']`. 

* Simple Imputer `['EnclosedPorch', 'WoodDeckSF']` Have a high percentage of missing values, but in real life not all houses do have a porch nor a wood deck area so this will be filled with 0. 


**The list above is your guide, your map to know at which stage you are in the data-cleaning process**

### Split Train and Test Set

In [ ]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

In [ ]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

In [8]:
median_imp =  ['2ndFlrSF', 'BedroomAbvGr', 'GarageYrBlt', 'LotFrontage', 'MasVnrArea']
drop = ['EnclosedPorch', 'WoodDeckSF',]
missing_cat = ['BsmtFinType1','GarageFinish']

---

### MeanMedianImputer

**Impute missing values with median** 

Since the missing data levels isn't so high 
- `['2ndFlrSF', 'BedroomAbvGr', 'GarageYrBlt', 'LotFrontage', 'MasVnrArea']`

In [ ]:
variables_method = median_imp
imputer = MeanMedianImputer(imputation_method='median', variables=variables_method)
df_method = imputer.fit_transform(TrainSet)

DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

### Categorical Imputation

**Imputation method 'missing'**
- `['BsmtFinType1','GarageFinish']`

In [9]:
variables_method = missing_cat
imputer = CategoricalImputer(imputation_method='missing', fill_value='No Record', variables=variables_method)
df_method =imputer.fit_transform(TrainSet)

DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

NameError: name 'TrainSet' is not defined

The distribution has been impacted as thought but for now i will apply this changes to prepare the model for data similar to the real-time data, 
from my research real estate house data is usually right skewed towards the higher prices

* In this case, there is no effect on the distribution of the variable, since you are not removing rows, but columns.
* The effect might be losing features that might have a relevant impact on your machine-learning model.

* Step 5: If you are satisfied, apply the transformation to your data.

In [10]:
from feature_engine.selection import DropFeatures
pipeline = Pipeline([
      ('Median', MeanMedianImputer(imputation_method='median', variables=median_imp) ),
      ('CategoricalImputer', CategoricalImputer(imputation_method='missing', fill_value='No Record', variables=missing_cat)),
      ('Drop', DropFeatures(features_to_drop=drop))
])
pipeline

Pipeline(steps=[('Median',
                 MeanMedianImputer(variables=['2ndFlrSF', 'BedroomAbvGr',
                                              'GarageYrBlt', 'LotFrontage',
                                              'MasVnrArea'])),
                ('CategoricalImputer',
                 CategoricalImputer(fill_value='No Record',
                                    variables=['BsmtFinType1',
                                               'GarageFinish'])),
                ('Drop',
                 DropFeatures(features_to_drop=['EnclosedPorch',
                                                'WoodDeckSF']))])

### Data Cleaning Pipeline

### Fit pipeline

In [11]:
dff = pipeline.fit_transform(df)

In [ ]:
train = TrainSet.copy()
test = TestSet.copy()

In [ ]:
pipeline.fit(TrainSet)

TrainSet, TestSet = pipeline.transform(TrainSet) , pipeline.transform(TestSet)

**Evaluation**

In [ ]:
EvaluateMissingData(TrainSet)

# Push cleaned data to Repo

In [ ]:
dff.to_csv('outputs/datasets/cleaned/CleanedHousePricing.csv', index=False)

## Train Set

In [ ]:
train.to_csv("outputs/datasets/collection/raw/TrainRaw.csv", index=False)

In [ ]:
TrainSet.to_csv("outputs/datasets/collection/cleaned/TrainSet.csv", index=False)

## Test Set

In [ ]:
test.to_csv("outputs/datasets/collection/raw/TestRaw.csv", index=False)

In [ ]:
TestSet.to_csv("outputs/datasets/collection/cleaned/TestSet.csv", index=False)

Good job! Clear cell outputs

Well done! You can now push the changes to your GitHub Repo, using the Git commands (git add, git commit, git push)